
This repository is a tutorial dedicated to create a ML agent that can play chess in FHE using the Concrete-ML library. This work also [compete for a bounty program](https://github.com/zama-ai/bounty-program/blob/main/Bounties/Machine_Learning/create-an-app-play-chess-in-fhe.md).

# Problem description

Before digging into the code, we need to understand what the FHE means and how this will impact our model.

By using FHE, a user will first, locally, crypted a board of a chess game. This crypted data, will then be send to a server where an AI agent will play his turn. In this configuration, the server and the agent will receive a encrypted data. However, the server will not be able to understand the logic of it, but the agent does. Therefore, the IA agent will get this data, use it as an input and get back a move. This move will be encrypted and sent it back to the user, that can read and apply the agent move.

Through this constraint, we are limited in the proposed algorithm/model. As you may have already know, chess algorithm usually given a board, try to maximize the gain through the next possible move. So, given a position, an algorithm will iterate over all the possible moves, and choose the one that optimize the score of winning. However, the data sent are crypted. So, we do not know the position of the current model. Thus, we cannot guess what will be the possible movement allowed. This constraint our model that should 


# Get the data


First of all, for all Deep Learning project, we need data. We will use the lichess open librairy 
> https://database.lichess.org/#standard_games

We will have some data `.pgn.zst`




In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def data_train_test_split(filename: str) -> tuple[str, str]:
    train_filename = "/tmp/train.csv"
    test_filename = "/tmp/test.csv"

    df = pd.read_csv(filename)
    train, test = train_test_split(df, test_size=0.2)
    train.to_csv(train_filename)
    test.to_csv(test_filename)

    return train_filename, test_filename

In [3]:
from torch.utils.data import DataLoader

from chess_model.dataset import ChessDataset

train, test = data_train_test_split("./data/samples_data.csv")

training_chess = ChessDataset(train)
testing_chess  = ChessDataset(test)

batch_size   = 8
train_loader = DataLoader(training_chess, batch_size=batch_size, shuffle=True, num_workers=6)
val_loader   = DataLoader(testing_chess, batch_size=batch_size, num_workers=6)

In [4]:
inputs, classes = next(iter(train_loader))
classes


[tensor([[0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0.

In [5]:
inputs.shape

torch.Size([64, 12, 8, 8])

In [6]:
from chess_model.model import SimpleModel, ConvModel

model = ConvModel()

In [7]:
x, (y_piece, y_row, y_col) = next(iter(train_loader))

pred_piece, pred_row, pred_col = model(inputs)

criterion = nn.CrossEntropyLoss()


loss_p = criterion(pred_piece, y_piece)
loss_r = criterion(pred_row, y_row)
loss_c = criterion(pred_col, y_col)

loss = loss_p + loss_r + loss_c
loss


tensor(5.9509, grad_fn=<AddBackward0>)

In [8]:
pred_col.shape

torch.Size([64, 8])

In [9]:
pred_piece
torch.argmax(pred_piece, dim=1)

torch.argmax(y_piece, dim=1)

tensor([4, 2, 2, 1, 1, 2, 0, 5, 0, 4, 1, 1, 0, 3, 1, 5, 4, 0, 3, 5, 0, 0, 3, 0,
        4, 2, 5, 2, 3, 2, 4, 1, 4, 4, 0, 0, 0, 1, 4, 4, 1, 1, 3, 2, 3, 0, 5, 0,
        1, 1, 3, 2, 2, 3, 2, 2, 4, 3, 4, 5, 3, 0, 2, 0])

In [10]:
len(y_piece)

64

In [11]:
# accuracy_piece = int(torch.argmax(pred_piece, dim=1) == torch.argmax(y_piece, dim=1)) / len(y_piece)
# accuracy_piece

In [12]:
trainer = pl.Trainer(max_epochs=5)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name         | Type               | Params
-----------------------------------------------------
0  | conv1        | Conv2d             | 3.9 K 
1  | act1         | ReLU               | 0     
2  | pool1        | MaxPool2d          | 0     
3  | conv2        | Conv2d             | 11.7 K
4  | act2         | ReLU               | 0     
5  | pool2        | MaxPool2d          | 0     
6  | flat         | Flatten            | 0     
7  | fc3          | Linear             | 9.3 K 
8  | act3         | ReLU               | 0     
9  | fc_piece     | Linear             | 390   
10 | fc_row       | Linear             | 520   
11 | fc_column    | Linear             | 520   
12 | criterion1   | CrossEntropyLoss   | 0     
13 | criterion2   | CrossEntropyLoss   | 0     
14 | criterion3   | CrossEntrop

Epoch 0:  82%|████████▏ | 1886/2291 [00:23<00:05, 79.72it/s, v_num=40, train_loss=1.740, train_accuracy_piece=0.228, train_accuracy_row=0.177, train_accuracy_col=0.144]  